# Demo on using S-BERT for unsupervised automated fact-checking

How the fact-checking model works:
1. **Semantic Search**: based on input text (_claim_), model will search and retrieve the most relevant piece of FAQ (_fact_) as evidence
2. **Recognizing Textual Entailment (RTE)**: after evidence (_fact_) has been established, the model will perform RTE to determine if (_claim_) contradicts, entails or is undertermined to the (_fact_)

In [1]:
!pip install -U sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

### 1. Storing & Loading S-BERT Embeddings

The easiest method is to use pickle to store pre-computed embeddings on disc and to load it from disc. This can especially be useful if you need to encode large set of sentences.

FAQs Corpus Used: Subset of _Retirement_ + _Account Services (Nominations, Distribution of CPF)_

In [2]:
import warnings
warnings.filterwarnings('ignore') # setting ignore as a parameter

from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pickle
import gradio as gr

from ipywidgets import interact, widgets
from IPython.display import display

# Loading pre-trained transformer models from huggingface
sentence_similarity_model = SentenceTransformer('msmarco-distilbert-base-v4')
nli_model = CrossEncoder('cross-encoder/nli-deberta-base')
nli_model_v3 = CrossEncoder('cross-encoder/nli-deberta-v3-base')

# using FAQs answers as corpus of facts for evidence finding
corpus = ['A CPF nomination provides you with the option to specify who will receive your CPF savings, and how much each nominee should receive, upon demise. The nominee(s) will receive the CPF savings due to them in cash or via cheque. This is the default mode for CPF nomination unless you opt for a different type of CPF nomination.',
          'You should make a CPF nomination so that your loved ones can receive your CPF savings swiftly and conveniently upon your passing. A nomination also gives you the flexibility to decide how you want your CPF savings to be distributed and the proportion of your savings each nominee will receive. Without a nomination, your CPF savings will be distributed by the Public Trustee’s Office to the legally entitled beneficiaries (who are usually family members and next-of-kin) according to existing intestacy or Muslim inheritance laws. The Public Trustee’s Office charges a fee for the distribution of your un-nominated CPF savings.',
          'You can make a CPF nomination as long as you are a CPF member, above 16 years old and mentally sound.',
          'Your CPF savings are excluded from your estate and therefore cannot be covered under a Will. Excluding CPF savings from the estate protects your savings from any creditor claims on outstanding loans that you may have after your passing. This ensures that your CPF savings are preserved fully for your loved ones to claim them swiftly. You are encouraged to make a CPF nomination if you wish to distribute your CPF savings according to your wishes. Without a nomination, your CPF savings will be distributed by the Public Trustee’s Office to the legally entitled beneficiaries (who are usually family members and next-of-kin) according to existing intestacy or Muslim inheritance laws. The Public Trustee’s Office charges a fee for the distribution of your un-nominated CPF savings. Please visit pto.mlaw.gov.sg for more information.',
          'A will is generally a document in writing executed in accordance with prescribed formalities, which sets out a testator (the person making the will) instructions and wishes as to how his property and assets are to be distributed after his death. For information, persons who inherit or benefit under the will are known as "beneficiaries" and the "executor" is the person nominated by the testator to administer and distribute his estate upon his death.',
          'The Public Trustee is a government body that acts as a trustee of the deceased estate. It ensures that the assets of the deceased are distributed to the rightful beneficiaries.',
          'When you are no longer around, we will pay the savings in your Ordinary, Special, MediSave and Retirement Accounts to the Public Trustee\'s Office for distribution to your family member(s) in accordance with the intestacy laws or inheritance certificate (for Muslims) in Singapore if you did not make a CPF nomination.',
          'Member\'s CPF savings will be transferred to the Public Trustee\'s Office for distribution in accordance with the Intestate Succession Act (Cap 146) if he did not make a CPF nomination.',
          'When you are no longer around, we will pay your CPF savings to the Public Trustee\'s Office for distribution in cash to your family member(s) in accordance with the intestacy laws or inheritance certificate (for Muslims) in Singapore. This ensures that the welfare of your dependants is provided for.',
          'Your witnesses for your CPF nomination must be two different individuals of at least 21 years of age, be of sound mind, and must not be any of your nominee(s). If you are making a nomination online, your witnesses would also need to have a valid Singpass each.',
          'Your witness cannot be nominated to receive your CPF savings.',
          'The Board\'s Customer Service Executives can act as your witnesses if you\'re making your CPF nomination at CPF Service Centre.',
          'If your nominee becomes a bankrupt after you made a CPF nomination, and remains an undischarged bankrupt at the time he or she becomes entitled to receive your CPF monies ("nominated monies"), the Board is legally obliged to inform the Official Assignee ("OA") and transfer the nominated monies to the OA upon their instructions. This is by virtue of the laws in Singapore relating to bankruptcy which vest the property of a bankrupt in the OA upon the making of a bankruptcy order.',
          'You can nominate a person who is bankrupt as your nominee. However, if your nominee is an undischarged bankrupt at the time he or she becomes entitled to receive your CPF monies ("nominated monies"), the Board is legally obliged to inform the Official Assignee ("OA") and transfer the nominated monies to the OA upon their instructions. This is by virtue of the laws in Singapore relating to bankruptcy which vest the property of a bankrupt in the OA upon the making of a bankruptcy order.',
          'The Board will not disclose your CPF account information and nomination details while you are still alive. This information will only be disclosed upon your demise, to the person(s) you have authorised in your nomination form.',
          'Upon your demise, CPF Board will disclose the following information to your authorised person(s) upon his/her request: The names of your nominee(s) and witnesses The relationship between you, your nominee(s) and witnesses when making the nomination The proportion of your CPF savings that each nominee has been nominated to receive',
          'CPF Nomination covers all savings in your MediSave Account, including those in your Ordinary, Special and Retirement Accounts.',
          'CPF members who have attained 16 years of age are able to make a CPF nomination. The introduction of the minimum age for making a CPF nomination is to be in line with the minimum age to work and earn wages under the Employment Act.',
          'You can instruct the Board to disclose your CPF account information and nomination details to an authorised person(s) upon your demise by completing the section “Instruction to Disclose my CPF Information” in the nomination form. The Board, upon request, will disclose your CPF statement(s) of account in and after the year of your death and your nomination details to your authorised person(s). We encourage you to authorise the disclosure to your nominee(s) so that he/she can reconcile your CPF account information when receiving your CPF savings (upon your demise).',
          'Divorce does not revoke a CPF nomination. This treatment is similar to that for wills, where a divorce does not revoke a will. After divorce, some CPF members may still wish to bequeath their CPF savings to their ex-spouse and children via their existing nominations, and therefore, it is best for the members to review their latest personal circumstances and update their nomination accordingly if required. Members can make their nomination online at my cpf Online Services.',
          'Your CPF nomination will be revoked upon marriage. When you get married, you will form a new family nucleus comprising your spouse and possibly children in the future. Your nomination is revoked upon marriage so that you can reconsider your bequeath intentions based on your new family nucleus. This practice is similar to Wills, which will also be revoked upon marriage under the Wills Act. You are strongly encouraged to make a new CPF nomination after your marriage if you want your CPF savings to be distributed according to your wishes. I have made a CPF nomination. Am I required to notify the CPF Board when I get married?',
          'You will not be able to indicate your nominee\'s share of your CPF monies as a fixed amount because if there are several nominees and the CPF balance is insufficient at the time of death, the Board would not be able to make payment based on the member\'s intention. As such, the nominee\'s entitlement to the CPF monies should be indicated as a percentage and the total of such entitlement must be 100%.',
          'You can indicate any person or organisation as a nominee in your CPF nomination. This includes non-CPF account holders. If you are making your nomination online, and if your nominee is a Singaporean or a Singapore Permanent Resident, do enter his/her name exactly as per the NRIC. If you face any issue, please email nomination@cpf.gov.sg for assistance.',
          'Yes, you will still need to make a CPF nomination as CPF savings are excluded from your estate and therefore cannot be covered by a will. Excluding CPF savings from your estate protects your savings from any creditor claims on outstanding loans that you may have after your passing. This ensures that your CPF savings are fully preserved for your loved ones. You are encouraged to make a CPF nomination to distribute your CPF savings according to your wishes.',
          'We encourage you to make a CPF nomination as soon as you begin to accumulate savings in your CPF account. As you progress in life, you should also review your nomination regularly, especially after major life events (marriage, childbirth, divorce etc.) so that your nomination continues to fit your intention.',
          'By making a CPF nomination, you have the flexibility to decide who should receive your CPF savings  upon your passing. Your nominee(s) will also receive your CPF savings more quickly (within five weeks), with no fees charged. If you do not make a CPF nomination, the Public Trustee’s Office (PTO) will distribute your CPF savings according to intestacy or Muslim Inheritance laws. Under this process, beneficiaries will need to provide documentation to prove their relationship with you. As time and effort is required to identify and verify your eligible beneficiaries, the entire process may take up to six months and it will incur an administrative fee which will be deducted from your CPF savings. This means your beneficiaries will receive less of your CPF savings. With a CPF nomination, CPF Board will take less than five weeks to distribute your CPF savings to your nominee(s). Make a CPF nomination so that your CPF savings are swiftly distributed to your loved ones when you are no longer around.',
          'Upon your demise, your next-of-kin may want to know how much CPF savings you have accumulated, who you have nominated to receive them and how much. To protect your privacy, only your nominee(s) will be told the amount they will receive but they will not know how much CPF savings you have in total, nor who else you have nominated to receive your CPF. This protects your privacy but can sometimes result in unhappiness. To avoid this, you may choose to authorise specific persons to access your CPF account information upon your demise. Without this explicit authorisation, your account information and nomination decisions will not be revealed.',
          'You can designate authorised person(s) by completing the section “Instruction to Disclose my CPF Information” in the nomination form, when you are making a nomination. If you already have an existing nomination, you will need to make a new nomination to add in the authorised person(s) whom you would like to give access to your CPF account information after you pass away.',
          'The following information will be disclosed upon the request of the authorised person(s): the names of your nominee(s) and witnesses, the relationship between you, your nominee(s) and witnesses when making the nomination, the proportion of your CPF savings that each nominee has been nominated to receive, and your CPF statements of account in and after the year of your death.',
          'For online nominations, you can specify any or all of your nominees, and up to five persons who are not your nominee(s) as your authorised person(s).',
          'Yes, you can do so provided you are conscious and of sound mind when making the nomination. You can consider appointing a doctor to be one of your witnesses in your nomination. You can request your doctor to write a memo stating that you have the mental capacity to make a CPF nomination and submit it to us.',
          'Yes, the SMS is legitimate. From 1 Aug 2022, the Board will use SMS sender ID, “CPF Board”, to send messages pertaining to your CPF, Workfare and Silver Support matters. You received the SMS because you have a registered mobile number with the Board. You may check or update your mobile number with your Singpass.',
          'The use of a single SMS sender ID \'CPF Board\' allows you to easily verify the legitimacy of messages sent by the Board. Only the Board can send SMSes using this ID.',
          'The Board has registered the new SMS sender ID with the Singapore SMS Sender ID Registry (SSIR), a setup by Infocomm Media Development Authority (IMDA). The SSIR can identify and block spoofed messages upfront, and prevent scammers from pretending to be CPF Board.',
          'We will no longer send SMSes from “SG-Workfare” and “SG-SSS” from 1 Aug 2022. You will only need to take note that the Board will use a new SMS sender ID, “CPF Board”, to send messages pertaining to your CPF, Workfare and Silver Support matters.',
          'We have standardised all SMSes sent to use the sender ID of “CPF Board”. Please check that you have registered your mobile number with us at the CPF website.',
          'The new sender ID “CPF Board” will be in service only from 1 August 2022. The SMS is meant to provide an advance notice to all CPF members before the Board starts to use the new sender ID “CPF Board”.',
          'As part of the transition to the new sender ID, members who perform online transactions or interact with the Board may start receiving SMSes from “CPF Board” as early as 31 Jul 2022.',
          'We will no longer send SMSes from “CPFBOARD”, “SG-Workfare”, “SG-SSS” from 1 Aug 2022. You may save the previous messages for your personal reference or delete them as you wish.',
          'There are specific phone models which automatically remove the spaces in sender ID. It mainly affects Apple users with iOS version 12 and below. You can be assured that all SMSes appearing in the thread under “CPFBoard” are legitimate messages sent by the Board.',
          'From 1 August 2022, CPF Board will send SMSes using only the sender ID “CPF Board” on matters pertaining to their CPF, Workfare and Silver Support. We will stop using sender IDs “SG-Workfare” and “SG-SSS”.',
          'Generally, we will contact the nominee(s) within 15 working days from the day we are notified of the member\'s passing, to provide them with details on the withdrawal process.',
          'Unclaimed CPF savings will continue to earn base interest in the respective accounts for up to 6 months. The CPF savings will not earn extra interest once we are notified of the member\'s death. Nominee(s) continue to have the right to claim the nominated savings at any time.',
          'The deceased member\'s CPF LIFE premium balance (if applicable) will be paid to his nominee(s) together with his CPF savings.',
          'Yes, the deceased member’s entire MediSave can be used to pay the last hospital bill if he had signed the Medical Claims Authorisation Form (MCAF), which authorises the use of the deceased member’s MediSave to pay the hospitalisation bill, before his passing. Any remaining MediSave after the payment of the last medical bill will be distributed to the deceased member’s nominee(s) if he had a valid nomination. This is provided that the MediSave deduction is completed before the distribution of the deceased member’s CPF balances.',
          'No, the deceased member’s CPF savings will not be used to pay his income tax liabilities. This is because CPF savings do not form part of the deceased member’s estate.',
          'CPF Investment Scheme (CPFIS) investments are not covered under CPF nomination. When a member passes away, his CPFIS investments* and any cash balance in his Investment Account will form part of his estate. The estate administrator/executor can contact the respective product providers or agent bank to claim the investments and cash balance. These investments, cash balance and death benefits may be used to settle the deceased member’s debts in accordance with the Probate and Administration Act. Except for insurance policies where the member had a valid nomination. The nominated beneficiaries can contact the insurance company to claim the death benefits.',
          'The CPF savings used to buy the property need not be refunded to the deceased member’s CPF account. The property will form part of his estate if it is held in tenancy-in-common or passed on to the remaining surviving owner(s) if it is held in joint tenancy.',
          'If the deceased member’s CPF savings had been used under the CPF Education Loan Scheme for himself or someone else, the withdrawn monies, including accrued interest, do not have to be refunded to his account.',
          'If the deceased person was a Singapore Citizen or Permanent Resident, there is no need to inform CPF Board of the death. If the deceased person was a foreigner with a CPF account, you will have to inform CPF Board of his death.',
          'You can report the death of a foreigner who was a CPF member by post/email or by making an appointment to visit our CPF Service Centre, with the following documents: i. The deceased member’s original death certificate and current passport duly cancelled due to his passing, or a copy certified by an official from a Singapore Overseas Mission with their official seal/stamp ii. A letter from the relevant authority confirming the deceased member’s identity number, if the death certificate does not show it iii. Evidence of your relationship with the deceased member iv. Your identity card or passport, and contact details',
          'To better protect you from scams, we will only accept bank accounts where the bank is able to provide automated bank account verification to CPFB. This ensures that payment will only be made to a bank account belonging to you. Currently, only POSB, DBS, OCBC and UOB are able to provide this service. We welcome any other bank to set up similar arrangements with us.',
          'Your share of the deceased member\'s CPF savings will be paid to you via Inter-bank GIRO.',
          'You are encouraged to claim the deceased member\'s CPF savings as early as possible. You can do so anytime by submitting an online application.',
          'If you are an undischarged bankrupt at the time you become entitled to receive the nominated monies, we are legally obliged to inform the Official Assignee ("OA") and transfer the nominated monies to the OA upon their instruction. This is by virtue of the laws in Singapore relating to bankruptcy, which vest the property of a bankrupt in the OA upon the making of a bankruptcy order. If the deceased member held any discounted Singtel shares, they would be sold upon the OA’s instruction.',
          'The CPF savings would form part of the deceased nominee\'s estate and would be paid to the Executor/Administrator appointed under the Grant of Probate (GP)/Letters of Administration (LA). If there is no Executor/Administrator appointed, the CPF savings may be paid to a proper claimant if the sum does not exceed $50,000.',
          'A proper claimant is a beneficiary, usually a family member, of the deceased member/nominee. The proper claimant can claim monies due to the deceased member/nominee\'s estate via an application process in the scenarios as stated below, where individual items do not exceed $50,000: Dividends, sale proceeds and other monies in respect of discounted Singtel shares belonging to a deceased member who passed away before 1 January 1996 or did not make a nomination, and who does not have any personal representatives; or Nominated CPF savings that would have been paid to the deceased nominee who had passed away before he could receive it. For more details, please write to us.',
          'The deceased member’s CPF savings would be paid to a Donee or Deputy appointed to act on the nominee\'s behalf under the Mental Capacity Act.',
          'You will first need to obtain a presumption of death certificate, which declares the person as presumed to be dead, from the Court. This is in line with industry practice where a person is treated as alive until the person has been officially reported as deceased. For any financial assistance in obtaining the presumption of death certification from the Court, you can contact the Legal Aid Bureau.',
          'Singapore Overseas Missions located in certain countries can help CPF members residing overseas in witnessing and certification of applications and documents. You will need to visit the Singapore Overseas Mission in person. Find out where is the overseas mission nearest to you.',
          'The Retirement Account is to provide you with monthly payouts from your payout eligibility age to supplement your retirement needs. It is created when you reach 55.',
          'When you turn 55, we will transfer your Special Account (SA) and Ordinary Account (OA) savings, up to your Full Retirement Sum (FRS), to create your Retirement Account (RA). Your SA savings will be transferred first. You do not need to top up your RA if you do not have enough savings to make up your FRS. Instead, if you have used CPF savings for your property, we will use your CPF savings withdrawn for your property (including accrued interest) to meet your FRS. The maximum amount that can be used is your Basic Retirement Sum. When you sell your property, you will have to restore your RA up to your FRS with your sales proceeds.',
          'Generally, the interest paid on funds depends on its tenure. Long-term funds earn higher interest compared to short-term funds. Hence, Special Account (SA) savings which earn higher interest compared to Ordinary Account (OA) savings are transferred to Retirement Account first to be streamed out as retirement payouts in the future. Did you know? If you wish to earn higher interest and boost our monthly payouts, you can transfer additional savings in your SA, followed by savings in your OA (which earns a base interest of 2.5%), to your RA (which earns a base interest of 4%), or make a cash top-up to your RA, up to the current Enhanced Retirement Sum.',
          'No, you do not need to top up your Retirement Account (RA) if you are unable to set aside your Full Retirement Sum (FRS). You will receive your monthly payouts in retirement and the amount you will receive will depend on your RA balances. Find out how your FRS is set aside when you turn 55. With a lower RA balance, you will receive lower monthly payouts. If you wish to receive higher monthly payouts, you can consider topping up your retirement savings. Use the CPF LIFE Estimator to estimate how much you need to top up to reach your desired monthly payouts. However, if you receive any new contributions, government grants or other refunds in your CPF accounts after 55, part or all of these amounts will be transferred to your RA up to your FRS (or Basic Retirement Sum if you own a property) on your next withdrawal. This ensures that you receive sufficient payouts for your retirement needs.',
          'No, only your Special Account (SA) and Ordinary Account (OA) savings will be used to make up your Full Retirement Sum (FRS). If you do not have enough SA and OA savings, we will use your CPF savings withdrawn for your property (including accrued interest) to meet your FRS. The maximum amount that can be used is up to your Basic Retirement Sum. Find out how your FRS is set aside when you turn 55.',
          'The Basic Retirement Sum (BRS), Full Retirement Sum (FRS), and Enhanced Retirement Sum (ERS) serve as guideposts in helping you set aside savings for your desired retirement payouts. The BRS is meant to provide you with monthly payouts in retirement that cover basic living expenses. If you own a property in Singapore with remaining lease that can last you to at least 95 years old, you can set aside the BRS. The FRS is two times of BRS, and is meant to provide you with higher monthly payouts that cover rental expenses as well. If you don\'t own a property, or wish to receive the full monthly payouts, you can set aside the FRS. The ERS is three times of BRS. If you wish to receive even higher payouts, you can choose to set aside the ERS by making a top-up. See what are the retirement sums applicable to you. How much you set aside depends on your CPF balances and desired monthly payouts. Find out how much savings you need to set aside for your desired payouts.',
          'The Basic Retirement Sum (BRS) takes reference from a person\'s expenditure from a lower-middle retiree household, according to the latest Household Expenditure Survey (HES). It assumes that the member owns a property that can last him up to 95 and he does not need to pay rent in retirement. The Full Retirement Sum and Enhanced Retirement Sum are set at two and three times of the BRS respectively. The retirement sum you need to set aside depends on when you turn 55 and is fixed for the rest of your life. Find out why the retirement sums are adjusted yearly for successive cohorts.',
          'No, the interest earned on your retirement savings does not form part of the retirement sum that you are required to set aside at 55.',
          'The retirement sums are increasing yearly as future cohorts of Singaporeans will need to set aside more for retirement due to inflation, increasing standard of living and longer life expectancy. If you have already turned 55, the yearly increase in retirement sums do not affect you. The retirement sum you need to set aside depends on when you turn 55 and is fixed for the rest of your life. Did you know? If the Full Retirement Sum had remained unchanged from $30,000 when it was first introduced in 1987 (as the Minimum Sum), it would only provide a monthly payout of about $300 today. That would be insufficient to meet day-to-day living expenses and pay for rent. In comparison, the Full Retirement Sum of $192,000 in 2022 will provide a monthly payout of $1,470-$1,570 for life.',
          'You will receive a letter from CPF Board a few months before your 55th birthday, providing you with more information. The letter will also provide details on how you can make an appointment to attend the CPF Retirement Planning Service (CRPS), which is a one-to-one session that uses personalised information to explain the CPF rules which will affect members when they turn 55. Alternatively, you may view your personalised retirement planner online*. Finally, you may attend our talks to find out more.  *Only available starting from six months before your 55th birthday',
          'Your payout eligibility age is the age where you can start receiving your retirement monthly payouts. The current payout eligibility age for members born in 1954 and later is 65. This means that you can instruct us to start your monthly payouts anytime from 65 to 70. The table below shows the payout eligibility age for members born before 1954. Year of birth Payout eligibility age Before 1944 60 1944 to 1949 62 1950 and 1951 63 1952 and 1953 64 Did you know? The payout eligibility age is not linked to the retirement/re-employment age. Read more.',
          'The CPF payout eligibility age is not linked to either the retirement or the re-employment age. Today, you can start your CPF payouts at any time from age 65. There is no link to the current retirement age of 62 or the re-employment age of 67. Even if the retirement age and the re-employment age are raised, the CPF payout eligibility age remains unchanged. Find out more about the retirement and re-employment ages and the difference between the retirement age and re-employment age.',
          'We do not assume every member wants to start their payouts at 65, especially since there are benefits to starting payouts later. Your CPF savings earn interest of up to 6% per annum. Hence, the later you start your payouts, the larger your retirement savings will grow into. This increase in retirement savings will provide you with higher monthly payouts under CPF LIFE. Starting payouts automatically at 65 means denying members these benefits. This is why it has been a long-standing practice for members to instruct us when you wish to start your payouts. You are in the best position to decide based on your individual circumstances. Did you know? For every year deferred, your CPF LIFE payouts increase by up to 7%. For 5 years of deferral, your CPF LIFE monthly payout could increase by up to 35%.',
          'The CPF Lifelong Income For The Elderly (CPF LIFE) is a national longevity insurance annuity scheme that insures you against running out of your retirement savings, by providing you with a monthly payout no matter how long you live. Unlike investment products which are meant to increase our wealth, insurance products help to protect us against life uncertainties. CPF LIFE addresses the uncertainty of us outliving our savings by providing us with lifelong payouts.',
          'Nobody can predict how long anyone will actually live. Therefore, planning how much of our retirement savings to use for living expenses is a dilemma – Spend too much and we might not have enough if we live longer than we thought; spend too little and we might deprive ourselves from fully enjoying our hard-earned savings. With increasing advances in technology and healthcare, we are likely to live longer than our grandparents’ or parents’ generation. Therefore, we cannot predict how long we will live by looking at how long our parents or grandparents live. Otherwise, we may underestimate our lifespan and find our savings depleted with many more years to live when we are no longer able to work. With CPF LIFE, you do not need to worry. Through providing lifelong payouts, CPF LIFE gives you certainty by insuring you against running out of your retirement savings no matter what age you live to.',
          'It depends on a few factors – your citizenship, when you are born and the savings you have in your Retirement Account (RA). We will inform you before you turn 65 if you are automatically included under CPF LIFE. In general, you will be automatically included if you are a Singapore Citizen or Permanent Resident born on 1 January 1958 and after*, and have at least $60,000 in your CPF retirement savings when you start your monthly payouts. *If you are born between 1 January 1958 and 30 April 1961 and had at least $40,000 in your RA when you reach 55, you would have to join CPF LIFE.',
          'We will inform you before you turn 65 whether you are automatically included under CPF LIFE. If you are automatically included, you may not be exempted from CPF LIFE unless you have a pension or commercial annuity that pays the same or higher monthly payout as CPF LIFE. For more information, please refer to the conditions for withdrawing your retirement savings.',
          'You can choose to join CPF LIFE as long as you are a Singapore Citizen or Permanent Resident at any time from 65 to one month before you reach 80.',
          'There is no minimum amount of CPF savings required for joining CPF LIFE. However, if you join with a low Retirement Account (RA) balance (which is used to pay for the premium for your CPF LIFE plan), you may get a low monthly payout which may not be meaningful for your retirement. You can consider topping up your RA and use the CPF LIFE Estimator to estimate your monthly payout to see if it is worthwhile doing so before you join CPF LIFE.',
          'The maximum amount of CPF savings that you can use to join CPF LIFE is the prevailing Enhanced Retirement Sum (ERS), a sum set at three times the Basic Retirement Sum, plus any accumulated interest in your Retirement Account.',
          'There are three CPF LIFE plans available – Escalating Plan, Standard Plan and Basic Plan. It is important to plan for your retirement with inflation in mind. Assuming you receive $1,000 of monthly payouts today, in 20 years’ time, you may need about $1,500 to buy the same items today. Which plan you choose depends on how willing you are to adjust your retirement lifestyle, as things become more expensive. Three CPF LIFE plans available  If you are worried about things being more expensive as the years pass, then you need a retirement income that increases every year. With monthly payouts that increase by 2% every year for life, the Escalating Plan protects you against rising prices so that you can generally keep your lifestyle in your retirement years. Read more about how the CPF LIFE Escalating Plan works. If you are willing to cope with rising prices over the years by buying less and living a more modest lifestyle, the Standard Plan provides level payouts. Payouts start higher than the Escalating Plan if you join with the same CPF LIFE premium, but will remain the same for the rest of your life and will eventually be lower than the Escalating Plan payouts. Read more about how the CPF LIFE Standard Plan works. If you don’t mind starting with lower monthly payouts which will be progressively lower later on, then the Basic Plan is good enough. The Basic Plan is a legacy plan carried over from the time CPF LIFE was first introduced in 2009. Compared to the Standard Plan which gives higher and level monthly payouts, Basic Plan payouts are lower and will get progressively lower when your combined CPF balances eventually fall below $60,000. Read more about how the CPF LIFE Basic Plan works. In all three plans, you will enjoy payouts no matter how long you live, your premiums will enjoy risk-free CPF interest rates of up to 6% per annum, and any remaining CPF LIFE premium balance will be given to your beneficiaries, together with your remaining CPF savings when you pass on. In other words, you and your loved ones will always get back the premium that you have put into CPF LIFE.',
          'CPF LIFE is attractive for the following reasons: Payouts for life CPF LIFE provides you with retirement payouts no matter how long you live, unlike most private retirement income plans that only do so for a limited period. Safe product with high, risk-free returns CPF LIFE monies are guaranteed by the Singapore Government, and provide high, risk-free returns of up to 6% p.a. Annuities offered by the private sector are subject to investment market returns. Lower costs Being non-profit and administered by the CPF Board, the CPF LIFE scheme does not incur costs from advertising and agents’ commissions. With a large member base, longevity risks and costs can also be better spread out.',
          'CPF LIFE payouts are not guaranteed, but they are designed to be stable. Any adjustments to CPF LIFE payouts are expected to be small and gradual. CPF LIFE is a self-sustaining insurance scheme where payouts are matched to premiums. Guaranteeing a minimum payout would require higher premiums.',
          'The CPF LIFE Escalating Plan provides payouts that increase by 2% each year to protect you against rising prices. Escalating Plan payouts start lower initially but will eventually be higher than Standard Plan and Basic Plan payouts. This helps you to generally keep your standard of living even as things become more expensive in your retirement years. To illustrate, a monthly payout that starts at $1,000 when you are 65 would reach approximately $1,500 by the time you are 85. The 2% increase will take place every year in the month that the first payout was made. Under this plan, all your Retirement Account savings will be deducted as CPF LIFE premium when you join CPF LIFE, which can be anytime from 65 to 70. Your CPF LIFE monthly payouts will first be paid from your CPF LIFE premium. When your CPF LIFE premium is depleted, you will continue to receive the monthly payouts from the interest that you and other CPF LIFE members have accumulated, no matter how long you live.',
          'CPF LIFE Escalating Plan payouts are not specifically matched to actual inflation in each year so that the increase in payouts are predictable and stable.',
          'The CPF LIFE Standard Plan provides level payouts. It is suitable for people who are willing to cope with rising prices in the years ahead by buying less and living a more modest lifestyle. Payouts start higher than the Escalating Plan if you join with the same CPF LIFE premium, but will remain the same for the rest of your life and will eventually be lower than Escalating Plan payouts. Under this plan, all your Retirement Account savings will be deducted as CPF LIFE premium when you join CPF LIFE. Your CPF LIFE monthly payouts will first be paid from your CPF LIFE premium. When your CPF LIFE premium is depleted, you will continue to receive the monthly payouts from the interest that you and other CPF LIFE members have accumulated, no matter how long you live.',
          'CPF LIFE Basic Plan is a legacy plan carried over from the time CPF LIFE was first introduced in 2009. Many members would find the Standard or Escalating Plan more suited to their needs. Unlike the CPF LIFE Standard Plan that gives higher and level monthly payouts, the monthly payouts under the Basic Plan are lower and will get progressively lower when your combined CPF balances eventually fall below $60,000. This is because the extra interest earned on the first $60,000 of your combined CPF balances is credited to the Retirement Account (RA) and paid as part of your monthly payouts. As balances fall due to payouts, the extra interest earned and subsequent payouts will decline as well. Under the LIFE Basic Plan, about 10-20% of your RA savings will be deducted as CPF LIFE premium when you join CPF LIFE, which can be any time from 65 to 70. Your monthly payout will first be paid from your RA and is estimated to last until 90. Thereafter, monthly payouts will be paid from your CPF LIFE premium. If your CPF LIFE premium is depleted, you will continue to receive monthly payouts no matter how long you live.',
          'When CPF LIFE Basic Plan was first introduced in 2009, it was better than the previous default monthly payout scheme which did not have payouts for life. However, to better suit Singaporeans’ needs, we have since introduced Standard and Escalating plans. Basic Plan is kept as a legacy plan. Monthly payouts under the Basic Plan start lower than Standard Plan and will get progressively lower when your combined CPF balances eventually fall below $60,000. If you are on the Basic Plan or other legacy plans such as the Balanced, Plus and Income plans, you can consider switching to the Escalating Plan to benefit from increasing payouts or the Standard Plan to enjoy higher and level payouts. Simply make the switch before age 80.',
          'If you are on any of the above legacy plans, you can switch to Escalating Plan to benefit from increasing payouts or Standard Plan to enjoy higher and level payouts. Simply make the switch before age 80.',
          'The CPF LIFE premium and payout levels are determined by an independent actuarial consultant. They are dependent on factors such as: Gender; Age; CPF interest rates; and Mortality rates',
          'CPF LIFE premiums are invested in Special Singapore Government Securities (SSGS), which are safe instruments because they are issued and guaranteed by the Government, and shielded from financial market fluctuations.',
          'Your CPF LIFE premium will continue to earn 4% base interest and up to 2% extra interest (on the first $60,000 of your combined CPF balances) like the Retirement Account savings of all other CPF members. The interest earned is factored into the higher monthly payout which you will enjoy. For more information, please refer to How interest is earned on CPF balances.',
          'You will benefit from the interest earned in two ways: It is factored into your monthly payouts and you receive a higher payout from the start. The interest accumulated on your CPF LIFE premium (and premiums of other CPF LIFE members) is what allows you to continue receiving payouts even after your CPF LIFE premium balance is depleted.',
          'This is not relevant because CPF LIFE is not an investment product to maximise returns from. Rather, CPF LIFE is a national longevity insurance annuity scheme which protects you against the possibility of having nothing to live on when your retirement savings are used up. If you pass away before your CPF LIFE premium is depleted, your beneficiaries will always get back any remaining CPF LIFE premium that you have put into your plan.',
          'You can use the CPF LIFE Estimator to estimate the amount of Retirement Account savings you need, based on your desired monthly payout and choice of payout start age.',
          'In general, you will be automatically included under CPF LIFE if you are a Singapore Citizen or Permanent Resident born on 1 January 1958 and after*, and have at least $60,000 in your CPF retirement savings when you start your monthly payouts. If you are a Singapore Citizen or Permanent Resident and not automatically included in CPF LIFE, the latest you can join the scheme is one month before you turn 80. * You will be informed before you turn 65 whether you will be automatically included in CPF LIFE. If you are born between 1 January 1958 and 30 April 1961 and had at least $40,000 in your Retirement Account when you reach 55, you would already been included under CPF LIFE.',
          'You will only need to choose your CPF LIFE plan when you wish to start receiving your CPF LIFE monthly payouts. If you are automatically included in CPF LIFE, this can be any time from 65 to 70. CPF Board will notify you before you reach 65 to explain the options you have and the choices you have to make. If you do not choose a plan before age 70, we will automatically place you on the CPF LIFE Standard Plan and start your payouts at 70. If you are not automatically included in CPF LIFE, you can choose your CPF LIFE plan when you wish to join the scheme, any time from 65 to one month before you turn 80. For more details on how to decide on the right CPF LIFE plan for you, please refer to What are the CPF LIFE plans available and which is the right plan for me?',
          'You will not be able to change your CPF LIFE plan as this will affect the payouts of other members on the scheme. However, if you change your mind within 30 days from the date of your policy letter, you may make your request via My Mailbox. If you are on a legacy plan such as the Basic, Balanced, Plus or Income Plan, you can switch to the Escalating Plan to benefit from increasing payouts or Standard Plan to enjoy higher and level payouts. Simply make the switch before age 80.',
          'All you have to do is to choose a CPF LIFE plan when you wish to instruct us to start payouts. If your application is received this month, your policy will be issued, latest by the following month. You will start receiving your payouts the following month after your policy is issued.',
          'You will receive the monthly payouts in your bank account on the 2nd or 3rd day of each month, if your bank supports Fast And Secure Transfer (FAST). Otherwise, you will receive your payouts later, via Inter-bank GIRO (IBG). If you wish to receive your monthly payouts earlier, you can update your bank account with CPFB to a FAST bank account via the CPF website.',
          'You can update your bank details for your CPF LIFE monthly payouts via the CPF website.',
          'There is no age limit for existing CPF LIFE member to increase their CPF LIFE premium.',
          'You can get higher CPF LIFE monthly payouts by: Topping up your Retirement Account (RA) under the Retirement Sum Topping-up Scheme You or your loved ones can make cash top-ups and CPF transfers to your RA up to the Enhanced Retirement Sum by following the steps below: 1. Head to cpf.gov.sg/rstuform 2. Log in using your Singpass The more RA savings you use to join CPF LIFE, the higher your monthly payouts. Starting your monthly payouts later (up to 70) You can start your payouts at a later age. For each year that you start later, your CPF LIFE monthly payouts will increase by up to 7%. This will give you up to 35% increase in payouts if you choose to start your payouts at 70 as your CPF savings will accumulate more interest. You will only need to apply for your CPF LIFE plan when you wish to start your payouts. The latest age you can start your payouts is 70, after which they will automatically begin. Monetising your property If you own a property, you can also consider monetising your property. You can use the CPF LIFE Estimator to find out how much RA savings you need, or the age you need to start your payouts to get your desired monthly payout.',
          'If you have already joined CPF LIFE, you can increase your monthly payouts by: Using your withdrawable CPF savings to increase your CPF LIFE premium If you are born in 1958 or after, you can withdraw up to 20% of your retirement savings from 65. This includes the first $5,000 withdrawable at any time from 55. If you are born in 1957, you may withdraw up to 10% of your retirement savings from 65, as you can already withdraw up to 10% of your CPF savings from 55. You can choose to use this withdrawable amount to increase your CPF LIFE premium. If you would like to do so, please write in to us via My Mailbox. Topping up and increase your CPF LIFE premium You or your loved ones can make cash top-ups and CPF transfers to your Retirement Account up to the current Enhanced Retirement Sum by following the steps below: 1. Head to cpf.gov.sg/rstuform 2. Log in using your Singpass Deferring your monthly payouts (up to 70) You can apply to defer your monthly payout by following the steps below: 1. Login to my cpf digital services with your Singpass 2. Click on My Requests > CPF LIFE > Apply to Defer CPF LIFE Payout Monetising your property If you own a property, you can also consider monetising your property. If you wish to enquire on your revised payout, please write in to us via My Mailbox.',
          'You may defer your CPF LIFE monthly payout up to 70. To defer your monthly payout, you can follow the steps below: Login to my cpf Online Services with your Singpass Click on My Requests > CPF LIFE > Apply to Defer CPF LIFE Payout',
          'You will not be able to defer your CPF LIFE monthly payouts, including any Additional Monthly Payouts (AMP), beyond 70. Both CPF LIFE payouts and AMP will have to start concurrently. We understand that some members think of their savings in CPF as an investment to enjoy the interest rates. However, the purpose of CPF savings is to provide members monthly payouts in retirement. Members who have no immediate use of their monthly payouts after 70 may invest them outside the CPF system if they wish.',
          'Your CPF LIFE monthly payout may be adjusted when our actuarial consultant reviews your CPF LIFE payouts yearly to take into account the following: Deviations in mortality experience. If more people live longer than expected in the long term, the monthly payouts might be lower, and vice versa. Deviations in interest rates. If interest rates are higher than expected, the monthly payouts might be higher, and vice versa. An annual review is necessary so as to ensure that CPF LIFE is sustainable over the long term. The revised payout will take effect from every July of the year if applicable. Any adjustments are expected to be small and gradual. You will be informed through a notification from us if your payout is revised.',
          'Outflows from your Retirement Account (RA) will reduce your CPF LIFE monthly payout, while inflows to your RA will be used automatically to increase your CPF LIFE premium which increases your CPF LIFE monthly payout. If you have started receiving your CPF LIFE monthly payouts, you will receive a notification from the Board the month after the outflow/inflow, informing you of any revision in your monthly payout. Note: Outflows include lump-sum withdrawals from RA or the usage of RA monies for housing. Inflows can be from any top-ups, funds transferred from other accounts to the RA, interest earned or refunds from selling property or investments.',
          'If you are on a legacy plan (i.e. Basic, Balanced, Plus or Income Plan) which was introduced at the start of the scheme in 2009, your monthly payouts will get progressively lower when your combined CPF balances eventually fall below $60,000. To avoid a drop in your monthly payouts, you may consider switching your CPF LIFE plan to Escalating or Standard Plan. To increase your monthly payouts, you may consider doing the following: Use your withdrawable CPF savings to increase your CPF LIFE premium Make a cash top-up and/or CPF transfers into your Retirement Account Defer your monthly payouts (up to age 70) Find out more about these options to increase your monthly payouts. For personalised details on how your monthly payout will change if you switch your CPF LIFE plan and/or enhance your payouts, please go to cpf.gov.sg/CRPSappt to book your 1-to-1 CPF Retirement Planning Service (CRPS) consultation session with us at any CPF Service Centres.',
          'For members born before 1957 and reached 65 before 2022: If existing CPF LIFE member:  To start payout: There is no need to apply to start your CPF LIFE payouts. Payouts will automatically be paid to you from the month you turn 65. To defer payout: Login to my cpf digital services with your Singpass Click on My requests > CPF LIFE > “Apply to Defer CPF LIFE Payout” For members born in 1957 and after, and reaching 65 in 2022 and after: Login to my cpf digital services with your Singpass Click on My requests > Retirement > "Plan my payout from age 65"',
          'We have recently launched a new PlanMyPayout service to help eligible members make important decisions on their monthly payouts. You can use this service, at earliest 3 months before you reach age 65. This service is currently only available for members born in 1957 and after. With this service, you can find out your estimated monthly payouts when you: start at age 65 or defer to a later age opt into CPF LIFE and/or convert your withdrawable CPF savings into monthly payouts. You can also use the same service to submit your instructions.',
          'To verify if the email/WhatsApp is legitimate, please check the following: The email is sent from do-not-reply@inform.cpf.gov.sg, and you are directed only to our official CPF website (link starting with cpf.gov.sg) The WhatsApp message is sent from “CPF Board”, our official business account with a green badge The Board will only send digital notifications to the email address/mobile number registered in our records. You may verify your contact details in my cpf digital Services - My Notifications with your Singpass.',
          'We will send you a notification a few months before your 65th birthday, guiding you to explore what your monthly payout options are. The notification will be sent to you via letter/ email/ WhatsApp.',
          'You can unlock the value of your property and increase your monthly payouts using one of the following options: If you are moving to a 3-room or smaller flat, you can supplement your retirement income by topping up your CPF Retirement Account through the Silver Housing Bonus scheme to boost your CPF monthly payouts, and receive a cash bonus of up to $30,000. You can sell part of your flat’s lease to HDB through the Lease Buyback Scheme and use the proceeds to top up your CPF Retirement Account to boost your CPF monthly payouts in retirement, while continuing to live in your flat. You can also receive a cash bonus of up to $30,000.',
          'You will continue to receive the Additional Monthly Payout (AMP). Only new Retirement Account inflows from November 2021 will automatically be used to increase your CPF LIFE premium so that you will receive higher CPF LIFE monthly payouts for life.',
          'As with other CPF LIFE members, your Retirement Account (RA) inflows will automatically be used to increase your CPF LIFE premium so that you will receive higher CPF LIFE monthly payouts for life. The cut-off age of 80 only applies to members who want to join CPF LIFE.',
          'To better protect members from scams, we will only accept bank accounts where the bank is able to provide automated bank account verification to CPFB. This ensures that payment will only be made to a bank account belonging to the member. Currently, only POSB, DBS, OCBC and UOB are able to provide this service. We welcome any other bank to set up similar arrangements with us.',
          'Once you have joined CPF LIFE, you can only cancel your CPF LIFE plan for the following reasons. You have a reduced life expectancy due to a medical condition, as certified by an accredited doctor*. Severe medical conditions that cause you to be permanently unfit for work or to lack mental capacity permanently can also be considered. You are about to leave / have left Singapore and West Malaysia permanently with no intention of returning for work or to live. You are a Malaysian citizen and have left Singapore permanently to live in West Malaysia. You are fully exempted from setting aside the retirement sum in your Retirement Account because you are receiving a monthly pension / annuity payout. Upon cancellation, you will receive a refund of your CPF LIFE premium balance. If your CPF LIFE premium has been fully paid to you as monthly payouts, there will be no remaining premium balance to refund. *The list of accredited doctors comprises doctors from public hospitals, polyclinics or specialists. If clarifications are needed, we may request a detailed medical report from your doctor from the accredited list. If your doctor is not from the accredited list, an independent assessment will be required from the Board\'s panel of doctors.',
          'When you pass away, we will pay your CPF LIFE premium balance (if any) together with any remaining CPF savings to your beneficiaries.',
          'Your beneficiaries are persons whom you have nominated to receive your CPF savings, including any CPF LIFE premium balance, after your death. If you have not nominated anyone to receive your savings, we will pay them to Public Trustee for distribution according to the intestacy laws. You can make a nomination online by logging in to my cpf Online Services with your Singpass.',
          'After you pass away, your beneficiaries will receive your CPF LIFE premium balance, which is the total CPF LIFE premium that you have paid less the total payouts you have received from your premium, together with any remaining CPF savings. For example, if you paid a CPF LIFE premium of $200,000 and pass away after receiving a monthly payout of $1,000 for ten months, we will pay your CPF LIFE premium balance of $190,000 (i.e. $200,000 - [$1,000 x 10 months]) together with any CPF savings to your beneficiaries. On the other hand, if you pass away before receiving any payouts, we will pay your CPF LIFE premium balance of $200,000, together with any remaining CPF savings to your beneficiaries.',
          'You can start your monthly payouts anytime from 65 to 70. Note: If you are born before 1954, you are past your payout eligibility age and can choose to start your payouts anytime. ',
          'The latest age you can start your monthly payouts is 70. Find out what happens if we don’t receive your instruction to start your monthly payouts.',
          'We will send you a notification a few months before your 65th birthday, guiding you to explore what your monthly payout options are. The notification will be sent to you via letter/ email/ WhatsApp.',
          'To verify if the email/WhatsApp is legitimate, please check the following: The email is sent from do-not-reply@inform.cpf.gov.sg, and you are directed only to our official CPF website (link starting with cpf.gov.sg) The WhatsApp message is sent from “CPF Board”, our official business account with a green badge The Board will only send digital notifications to the email address/mobile number registered in our records. You may verify your contact details in my cpf digital Services - My Notifications with your Singpass.',
          'We have recently launched a new PlanMyPayout service to help eligible members make important decisions on their monthly payouts. You can use this service, at earliest 3 months before you reach age 65. This service is currently only available for members born in 1957 and after. With this service, you can find out your estimated monthly payouts when you start at age 65 or defer to a later age opt into CPF LIFE and/or convert your withdrawable CPF savings into monthly payouts. You can also use the same service to submit your instructions.',
          'Yes, you can opt out of receiving CPF monthly payouts and withdraw your retirement savings in full if you have other sources of lifelong income.',
          'To better protect members from scams, we will only accept bank accounts where the bank is able to provide automated bank account verification to CPFB. This ensures that payment will only be made to a bank account belonging to the member. Currently, only POSB, DBS, OCBC and UOB are able to provide this service. We welcome any other bank to set up similar arrangements with us.',
          'If you are not automatically included in CPF LIFE and did not opt to join CPF LIFE to enjoy lifelong payouts, you will receive monthly payouts which will stop when your retirement savings run out. Your monthly payouts are calculated to last you up to 20 years, with the extra interest earned on your retirement savings extending the duration up to 90 years old. Find out why you should be concerned about outliving your retirement savings.',
          'We cannot predict how long we will live. Hence, it is not easy to decide how much we need for our retirement. Spend too much and you might not have enough. Save too much and you end up depriving yourself. With technology and healthcare becoming more advanced, we\'re also likely to live longer. Most people underestimate how long they will live and don’t set aside enough. CPF LIFE provides certainty of lifelong payouts, so you can enjoy peace of mind in your retirement years, knowing that you will have monthly payouts no matter how long you live. On the other hand, if you are not on CPF LIFE, your monthly payouts will stop when your savings run out.',
          'No, you cannot defer your monthly payouts beyond 70. The purpose of your retirement savings is to provide you with a stream of retirement income. If you do not have an immediate need for your monthly payouts, you can consider investing them outside the CPF system. If you have lower retirement savings and wish to have them last you to a later age, you can write to us for our assessment.',
          'If you are born in 1948 and after We will start your payouts automatically when you turn 70, and will pay them into your bank account in our records. If you have not given us a bank account before, we will write to you to ask for your bank account nearer your 70th birthday. If you are born before 1948 Your payouts will not start automatically. You will need to apply to start your monthly payouts.',
          'You can consider the following options to increase or prolong your monthly payouts from your retirement savings: Making cash top-ups and CPF transfers to boost your retirement savings. Starting your monthly payouts later (up to age 70) to grow your retirement savings with attractive interest rates of up to 6% p.a.. Note: If you have started your payouts, you can request to defer your payouts up to age 70. If you have not started your payouts, you do not need to submit a request. Joining CPF LIFE to enjoy lifelong monthly payouts no matter how long you live.',
          'If you are not on CPF LIFE, your monthly payouts are calculated to last you for up to 20 years. This takes into account the projected growth of your retirement savings at the current base interest rate of 4% p.a.. As Singaporeans live longer, more than half of Singaporeans who are 65 today are expected to live more than 20 years. To protect you against the risk of outliving your payouts, the extra interest paid by the Government (up to 2% per year) on your retirement savings is used to extend your payouts beyond 20 years, up to 90* or another five years, whichever ends later. How are monthly payouts calculated if I am not on CPF LIFE? *Up to age 90, or another five years, whichever ends later. Use this payout estimator to check how much monthly payouts you can receive from your retirement savings.',
          'If you have reached 65, you can check your monthly payout in your Retirement dashboard. If you have not reached 65, we will notify you three months before 65 regarding your monthly payout and the decisions that you need to make . Note: If you are born in 1958 or after, you may be automatically included into CPF LIFE to enjoy lifelong payouts.',
          'We will review your payouts monthly to take into account the changes to your retirement savings. Your payouts will generally be revised in the following month after the inflows or outflows are made to your retirement savings.',
          'If you are not on CPF LIFE, you will stop receiving monthly payouts once your retirement savings run out. However, you may still have funds in your Ordinary Account (OA) and Special Account (SA). The automatic disbursement ensures that you can continue to receive monthly payouts to supplement your retirement expenses, without having to apply or transfer their OA and SA savings to your Retirement Account. This brings greater convenience to members, minimising disruptions to their retirement payouts and lifestyle.',
          'When your retirement savings are depleted, we will automatically stream out your Special Account and Ordinary Account savings to minimise the disruption to your payouts.',
          'You can continue to withdraw your available Special Account and Ordinary Account savings based on the current withdrawal rules.',
          'You will benefit from the auto-disbursement of Ordinary Account and Special Account savings, by having the flexibility to use additional cash on hand for your daily living expenses or to service your monthly housing instalments.',
          'The CPF is designed to provide members with an income in retirement. Auto-disbursement of your Special Account and Ordinary Account savings is aligned with this objective. You have the flexibility to use additional cash on hand for your daily living expenses or to service your monthly housing instalments (if applicable). Members who want to retain the monies in their CPF accounts to earn higher interest should look for other options besides CPF.',
          'If your bank supports Fast and Secure Transfer (FAST), you will receive your monthly payouts in your bank account on the 2nd or 3rd day of the month. If your bank does not support FAST, you will receive your monthly payouts by the 4th working day of the month via Interbank GIRO. To receive your monthly payouts earlier, update your bank account to a FAST bank account by submitting an online application, or by following the steps below: Login to my cpf digital services with your Singpass Click on My Requests > Retirement > “Start Monthly Payouts”',
          'If you are born in 1957 and after You can use the new PlanMyPayout service to help you make important decisions on your monthly payouts. With this service, you can check your estimated monthly payouts when you: start at age 65 or defer to a later age; opt into CPF LIFE; and/or convert your withdrawable CPF savings into monthly payouts. You can also use the same service to submit your instruction to start monthly payouts. Note: This is a new service and is currently unavailable for members born before 1957. Eligible members can only access the service earliest three months before 65. If you are born before 1957 You can submit an online application to start your monthly payouts, or follow the steps below: Login to my cpf digital services with your Singpass Click on My Requests > Retirement > “Start Monthly Payouts”',
          'You can update your bank account for your monthly payouts by submitting an online application, or by following the steps below: Login to my cpf digital services with your Singpass Click on My Requests > Retirement > “Start Monthly Payouts”',
          'No, we do not accept instructions to start monthly payouts before you turn 65. This is because your circumstances and retirement needs may change from now until 65. You can submit your request to start monthly payouts three months before you turn 65, up to age 70. We will send you a notification with more details three months before you reach 65, to guide you in exploring your monthly payout options. ',
          'The minimum monthly payout you have to meet for you to withdraw all your retirement savings depends on your birth date and gender. Find out what happens if the monthly payout from your pension/annuity is less than the minimum monthly payout.',
          'You can withdraw your retirement savings if you have a pension or annuity that provides you with monthly payouts for life. The amount of retirement savings you can withdraw will depend on the monthly payout you receive from your pension/annuity. Find out more.',
          'The minimum monthly payout for you to withdraw all your retirement savings using your pension/annuity is computed based on the monthly payouts you can receive under CPF LIFE Standard Plan with your Full Retirement Sum.',
          'You will be able to withdraw a partial amount of your retirement savings if your pension/annuity monthly payout is less than the minimum monthly payout. Your remaining retirement savings will be streamed out as monthly payouts to complement the monthly payouts you receive from your pension/annuity.',
          'No, you cannot choose the withdrawal amount even if you have a pension/annuity that allows you to withdraw your retirement savings. We will compute how much you can withdraw based on the monthly payout you are receiving from your pension/annuity.',
          'Top-ups to your retirement savings are meant to build up your retirement savings and boost your retirement payouts for your living expenses, and cannot be withdrawn as cash in a lump sum or used for other purposes. In addition to earning attractive risk-free interest rates on your top-ups, you might have also benefited from tax relief. Allowing withdrawals of the top-ups and the accrued interest would defeat the purpose of the scheme and the incentives given.',
          'No, you cannot top up or receive top-ups to your retirement savings if you have already withdrawn your retirement savings using your pension/annuity. However, you can top up if you choose to join CPF LIFE.',
          'If you surrender your pension/annuity after you have withdrawn your retirement savings, the surrender value of your pension/annuity, up to your Full Retirement Sum with accrued interest, will be refunded to your Retirement Account. Any excess amount will be refunded to you in cash. Please note that you will not be able to use your annuity to obtain a loan as we need to secure the refund of the surrender value when you surrender your annuity.',
          'If you are approved to withdraw all your retirement savings using your pension/annuity, you can decide either to: i. terminate your CPF LIFE plan and receive a refund of the premium balance (if applicable); or ii. retain your CPF LIFE plan and receive monthly payouts for as long as you live.',
          'You can apply for withdrawal of your CPF retirement savings if you have a pension/annuity, as long as you are: i. 55 or above; and ii. currently receiving monthly payouts from your pension/annuity.',
          'Yes, you can use multiple policies to apply for withdrawal of your retirement savings as long as they provide you with a lifelong income.',
          'If you are approved to withdraw your retirement savings using your pension/annuity, your retirement savings will be paid to your bank account via Interbank GIRO (IBG). To better protect members from scams, we will only accept bank accounts where the bank is able to provide automated bank account verification to CPFB. This ensures that payment will only be made to a bank account belonging to the member. Currently, only POSB, DBS, OCBC and UOB are able to provide this service. We welcome any other bank to set up similar arrangements with us.',
          'We will pay your retirement savings to your bank account within seven working days after we have received all required information. We may require more time if we need more information/documents pertaining to your application, such as outstanding tax liabilities with IRAS.',
          'If you are approved to withdraw your retirement savings using your pension/annuity, you will receive a withdrawal advice by mail once we have processed the payment to your bank account. You can also check your statement by logging on to cpf.gov.sg.',
          'Your property can be an additional source of retirement income. It’s an option to consider if your CPF and personal savings are not enough to provide you with your desired monthly payouts in retirement. Find out more about the different ways you could monetise your property. ',
          'It depends on your needs and preferences. As with all important financial decisions, take the time to understand the nature of the monetisation option, and assess your situation and retirement needs before deciding if the option is suitable for you. If you own an HDB flat and do not mind moving to a 3-room or smaller flat, you can consider the Silver Housing Bonus scheme. If you prefer to live in your existing flat, you can consider the Lease Buyback Scheme. Under these two schemes, part of your net sales proceeds will be used to top up your Retirement Account and provide you with lifelong monthly payouts under CPF LIFE. If you own a private property, consider speaking with your financial institution or advisor on the available products in the market that enable you to monetise your private property to supplement your retirement income, while ageing in place. Lastly, you can consider renting out a spare bedroom, or your entire property if you have an alternative accommodation. If you own a HDB flat, find out more about renting out a flat/bedroom',
          'The Silver Housing Bonus scheme (SHB) enables you to supplement your retirement income if you’re moving to a 3-room or smaller flat. Part of your net sale proceeds will be used to top up your Retirement Account to boost your lifelong monthly payouts under CPF LIFE. You can also enjoy a cash bonus of up to $30,000. Find out more about the SHB scheme.',
          'The Lease Buyback Scheme (LBS) enables you to supplement your retirement income by selling part of your HDB flat’s lease while you continue to live in it. Part of your net sales proceeds will be used to top up your Retirement Account to boost your lifelong monthly payouts under CPF LIFE. You can also enjoy a cash bonus of up to $30,000. Find out more about the LBS.',
          'You can apply for the Lease Buyback Scheme or the Silver Housing Bonus scheme through HDB: Silver Housing Bonus scheme Lease Buyback Scheme',
          'Consider speaking with your financial institution or advisor on the available products in the market that allow you to monetise your private property to supplement your retirement income while ageing in place. Like all important financial decisions, take the time to understand the nature of the product, and assess your risk tolerance and retirement needs before deciding if the monetisation option is appropriate for you.',
          'The fact is, you can withdraw your CPF savings anytime after turning 55, in full or partially, as long as you meet the applicable withdrawal conditions. And with PayNow, you will get your money almost instantly!',
          'The CPF Housing Scheme enables CPF members to use only their CPF Ordinary Account (OA) savings to: Buy Housing and Development Board (HDB) flats. Buy or build private residential properties in Singapore. Please note that Special Account (SA) and Retirement Account (RA) savings are meant to provide members with a basic income during retirement and hence cannot be used for flat/property purchases.',
          'Please refer to the table for our service standards for processing applications relating to housing matters. Service Processing Time Commence CPF monthly instalment / use CPF savings for arrears payment, partial or full repayment of housing loan Online: Within five working days Revise / cease CPF monthly instalment for housing loan taken from banks/ financial institutions Online: Within one working day Make voluntary cash refund of CPF savings used for property Online: Within one working day (eNETS), almost instantly (PayNow QR and OCBC PayAnyone) Manual: Within seven working days',
          'You cannot transfer your Ordinary Account (OA) savings to your parents\' OA to help them pay their outstanding housing loan. You can only use your OA savings to service the housing loan if you are a co-owner of the property.',
          'You can find a list of law firms appointed by the Board (PDF, 0.1MB) to handle all legal matters required in processing your application to use your CPF savings to buy a property.',
          'CPF savings can be used to pay the stamp duty and survey fees. However, monthly service and conservancy charges, and other charges related to the use of the property, including taxes, cannot be paid with your CPF savings. As stamp duty is payable within 14 days from the date of the sale and purchase agreement or the date of acceptance of the option to purchase, you may need to use cash to pay the stamp duty first. Subsequently, you can apply for a one-time reimbursement from your CPF account together with your application to use your CPF savings to buy a property.* For completed properties, you should apply for the one-time reimbursement of stamp duty and CPF lump sum drawdown on completion at the same time. For properties under construction, you should apply for the one-time reimbursement of stamp duty on or before the date of the legal completion. If the stamp duty is paid by the developer of your property, you are not allowed to seek reimbursement of the stamp duty from your CPF savings. *If you are taking a HDB loan, please check with HDB on your eligibility to apply for the one-time reimbursement of stamp duty from your CPF account.',
          'You can use your Ordinary Account savings to pay for the following when you purchase a property: the transaction fee levied by the lawyer\'s appointed bank. the lodgement fees levied by Singapore Land Authority for the holding and disbursing of your CPF savings. For more information on these costs, please check with your lawyers.',
          'You can use your Ordinary Account savings to pay the upgrading costs incurred under the Housing & Development Board Upgrading Programmes and Town Council Lift Upgrading Programme.',
          'CPF savings cannot be used for renovation, improvement or repair work for your property.',
          'CPF savings cannot be used to pay your seller\'s stamp duty. The seller\'s stamp duty has to be paid with cash.',
          'CPF savings can only be used to buy properties in Singapore.',
          'You may appoint someone under the Power of Attorney to act for you.',
          'If you have already used your Ordinary Account (OA) savings towards the subject property, please view your Home ownership dashboard to find out your latest usage and usage limit for your property. If you are planning to buy a property, please confirm the following: https://www.cpf.gov.sg/member/faq/home-ownership/housing-scheme/how-much-cpf-savings-can-i-use-for-my-property-purchase',
          'You can use your Ordinary Account savings to pay for the legal fees incurred when you buy a property. Please apply through HDB if you are taking an HDB loan or your lawyer if you are taking a bank loan to use your OA savings to pay for the legal fee.',
          'The application process to use your CPF savings to pay for the recess area of your HDB flat will depend on the type of loan you have taken.',
          'There are two ways for you to pay for the recess area of your HDB flat. Submit an online application at HDB’s website with your Singpass. Upon logging in, look under ‘e-Services > Residential > Living in a HDB flat > Application to Purchase Recess Area. Your application will be processed within three working days upon the receipt of your request from HDB. Complete the CPF withdrawal form (PHS9) at HDB Branch. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system. Your application will be processed within three to five working days upon the receipt of your request from HDB.',
          'Please make an appointment to submit your request at any of our service centres. Do note that counter services at CPF Service Centres are available by appointment only.',
          'You will need to produce a valuation report if you want to use your CPF savings to purchase your property. This will help us decide the amount of CPF savings you can use for your property. For HDB flat, we will only accept the valuation determined by HDB that is dated no longer than three months from the date of application received by HDB. For private property, we will only accept a valuation report, conducted by a licensed valuer not more than three months from the date of purchase.',
          'Whether you can use the valuation report done by a bank’s valuer depends on the type of property you are buying.',
          'It is HDB\'s requirement that you can only use the value of the flat determined by HDB.',
          'You can use the valuation report done by a bank\'s valuer.',
          'The amount of CPF savings you can use for your property’s down payment depends on the type of your property you are buying and loan you are taking.',
          'If you are buying an HDB flat directly from HDB, you can use your Ordinary Account (OA) savings to pay for the initial 10% downpayment as well as the balance of the purchase price. You would have to pay for the option fee in cash first at the time of booking an HDB flat. If you are buying a resale HDB flat, you will have to pay for the deposit, i.e., option fee and option exercise fee, in cash.',
          'You can only use your Ordinary Account savings to pay for the property after you have paid the following in cash: minimum cash downpayment of at least 5% of the lower of the purchase price or the valuation price of the property at the time of purchase; and the portion of the purchase price that is above the market value of the property; and the lower of the purchase price or the valuation price of the property at the time of purchase, less the housing loan and approved CPF lump sum that will be paid towards the property; and option fee at the time of booking an HDB flat (if you are buying the flat directly from HDB) or option fee and option exercise fee (if you are buying a resale flat). You may refer to the Monetary Authority of Singapore (MAS) website for more information on the minimum cash downpayment that you have to pay.',
          'You can only use your Ordinary Account savings to pay for the property on completion of all legal documentation and after you have paid the following in cash: a minimum cash downpayment of at least 5% of the lower of the purchase price or the valuation price of the property at the time of purchase; and the portion of the purchase price that is above the market value of the property; and the lower of the purchase price or the valuation price of the property at the time of purchase less the housing loan and approved CPF lump sum that will be paid towards the property. You may refer to the Monetary Authority of Singapore (MAS) website for more information on the minimum cash downpayment that you have to pay.',
          'Check if you meet the eligibility criteria at HDB website.',
          'Check if you meet the eligibility criteria at URA website.',
          'It is important to find out how much Ordinary Account (OA) savings you have to finance your purchase. Your existing OA balance and your future monthly CPF contributions to your OA will help you decide an affordable home price. You can use our First home calculator to estimate a housing loan and property price based on your income and your ability to service the loan. You can check your CPF OA balance on Member dashboard by logging in at our CPF website using your Singpass. Otherwise, you may visit any of the CPF Service Centre.',
          'Work out the amount of cash savings you have and the amount of housing loan you can get.',
          'You can check with HDB or if you are buying a resale flat from the open market, you can check with the seller that he/she will be refunding his/her CPF principal amount withdrawn and accrued interest used towards the flat so that the CPF charge (on the seller) can be lifted.',
          'You can check with your lawyer to make sure your property is free of any charges (i.e., CPF charge, mortgage charge) upon completion.',
          'Please apply through HDB if you are taking an HDB loan or your lawyer if you are taking a bank loan to use your OA savings for the property.',
          'You must first set aside your Basic Retirement Sum in your CPF accounts to provide you with a monthly income to support a basic standard of living during retirement before any excess Ordinary Account savings can be used to purchase Studio Apartment or short-lease 2-room Flexi Flat.',
          'You can refer to the terms and conditions to use your CPF savings to buy a property.',
          'The Memorandum of Mortgage (PDF, 77KB) sets out the priority arrangement between the Board and the bank in respect of the refund of proceeds arising from a disposal of the property where the mortgagee bank has the first charge. The Memorandum of Mortgage has been registered with Singapore Land Authority (SLA) under the registration number: IA/991901M.',
          'The maximum amount of CPF that can be used will be pro-rated depending on the extent the remaining lease of the property can cover the youngest buyer to the age of 95. This will help buyers set aside CPF savings for their housing needs during retirement (e.g. a replacement property). Property buyers may use the CPF housing usage calculator to compute their allowable CPF usage (for both public and private housing).',
          'There will be no change to the amount of CPF that you can use to pay for your mortgage for your existing property as the changes will apply only to properties purchased from 10 May 2019.',
          'The new rules will apply to you if: - For HDB flats: Your date of application on the HDB website is from 10 May 2019. - For private properties: Your Option to Purchase or Sale & Purchase Agreement (in the absence of an Option to Purchase) exercised date is from 10 May 2019.',
          'The changes on the use of CPF for housing will apply to all properties purchased from 10 May 2019. For HDB flats, the date of purchase refers to the date of application on the HDB website. For private properties, the date of purchase refers to the date of the Option to Purchase or Sale & Purchase Agreement (in the absence of an Option to Purchase).',
          'The rules were updated to take into account the changing needs and higher life expectancy of Singaporeans. Instead of focusing on the remaining lease of the property, the rules will now focus on whether the remaining lease of the property can cover the youngest buyer until at least the age of 95. This will give Singaporeans more flexibility to buy a home for life, while safeguarding their retirement adequacy.',
          'The required lease coverage of at least 95 years recognizes the increasing life expectancy of Singaporeans and is aligned with existing HDB rules for 2-room Flexi and Lease Buyback Scheme, where buyers are required to purchase/retain leases that cover them until at least the age of 95. Nevertheless, buyers who do not meet the required lease coverage are still allowed to use CPF, but at a pro-rated amount to help buyers set aside CPF savings for their housing needs during retirement (e.g. a replacement property).',
          'The changes are intended to give Singaporeans more flexibility in buying a home for life, while safeguarding their retirement adequacy, regardless of whether they are purchasing public or private properties. This is no different from the previous CPF usage rules for shorter lease properties, which applied to both public and private properties.',
          'The CPF usage rules encourage families to buy properties with sufficient lease to cover all co-owners for life. Hence, CPF usage is capped based on the age of the youngest co-owner using CPF. When the youngest co-owner relinquishes ownership of the flat, CPF Board will update the CPF usage limit based on the age of the next youngest remaining co-owner and inform all co-owners in writing.',
          'We recognise that the previous MLP of 30 years for CPF usage contributed to anxieties on the saleability of older properties and have lowered this to 20 years. This is the same as the MLP for obtaining a HDB loan. There will still be a MLP of 20 years to ensure prudent use of CPF monies. Nevertheless, most properties today are still young and far away from this MLP of 20 years.',
          'The total monthly CPF deduction for all owners must not exceed the total monthly repayment amount approved by your financier.',
          'We will deduct from your Ordinary Account to pay the monthly instalments as requested by you. There is no fixed order of deduction as to which property will be paid first. Please ensure that you have sufficient funds in your CPF account for the monthly instalment deductions.',
          'We will process the refunds within: One working day when the funds and refund details are transmitted through the electronic link-ups we have with the Housing & Development Board or the banks Seven working days when the funds and/or refund details are transmitted to us by other means',
          'Type of property and loan taken Date of next deduction Housing & Development Board (HDB) flat (HDB loan) The deduction is on the Sunday between 6th and 12th (including both dates) of the month. HDB flat (Bank loan) The deduction is between the 11th and 14th (including both dates) of the month. Private property / Executive Condominium The deduction is on the first or last working day of the month, depending on the bank\'s due date. Note: If you are using your CPF savings to pay for the upgrading cost of your HDB flat, the deduction is on the Sunday, between the 6th and 12th (including both dates) of the month.',
          'You will not be informed if you do not have enough CPF savings for the monthly housing loan deduction. However, you can view the amount deducted for your housing loan by checking your CPF online statement.',
          'The Board does not impose a penalty for unsuccessful housing deductions.',
          'You can make a voluntary contribution to your CPF account, subject to the CPF Annual Limit. The voluntary contributions will be apportioned and credited to your Ordinary, Special and MediSave accounts, based on the CPF allocation rates. As only Ordinary Account savings can be used for housing repayments, you may consider paying the shortfall in cash to your bank (if you are taking a bank loan) or Housing & Development Board (HDB) (if you are taking an HDB loan) directly.',
          'We are not authorised to automatically deduct the monthly instalment shortfall from your co-owner\'s CPF account for your property.',
          'You will have to authorise your lawyer (by signing the Letter of Authorisation, Declaration, Consent & Agreement) to submit your application on your behalf if you wish to refinance the housing loan for your property. The following documents are required: 1. Final/latest statement from the existing financier; and 2. Letter of offer from the new financier stating the type and quantum of the new loan. Please note that your current CPF monthly instalment payment to the existing financier (if any) will be ceased upon the approval of the application submitted by your lawyer. For HDB flat owners, if you are intending to use your CPF savings for the monthly instalment payment but currently not covered under the Home Protection Scheme (HPS), you will have to submit the HPS application online using your Singpass after receiving our notification that your application has been processed. Your eligibility for an HPS cover is subject to approval and you being in good health. While we are processing your HPS application, you can proceed to use your CPF savings to pay your monthly housing loan instalments. For HDB flat owners who are existing HPS insureds or exempted from HPS, you may find out more : What will happen to my Home Protection Scheme cover if I have refinanced my housing loan?',
          'Yes, you can apply to use your CPF savings to pay the monthly instalment shortfall. However, the refund of the cash paid is subject to your financier’s approval. Hence, please check with your financier directly if your financier is able to refund the cash paid before you apply to use your CPF savings to make the payment to your financier.',
          'The application process to use your CPF savings for your housing loan monthly instalments depends on the type of your property and loan you have taken.',
          'There are two ways for you to commence your housing monthly deduction. Submit an online application at HDB’s website with your Singpass. Upon logging in, look under ‘My Flat > Purchased Flat > Financial Info > Other Related Services > Commence/Vary Monthly CPF deduction’. Your application will be processed within three working days upon the receipt of your request from HDB. Complete the CPF withdrawal form (PHS9) at HDB Branch. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system.  Your application will be processed within three to five working days upon the receipt of your request from HDB. Note: You have to be insured under Home Protection Scheme (HPS) if you are using your CPF savings to pay your housing loan monthly instalments for HDB flats. Your eligibility for an HPS cover is subject to approval and you being in good health. While we are processing your HPS application, you can proceed to use your CPF savings to pay your monthly housing loan instalments.',
          'You can commence your housing monthly deduction by submitting an online application with your Singpass. Please submit your application at least 5 working days before monthly instalment deduction date. Note: You have to be insured under Home Protection Scheme (HPS) if you are using your CPF savings to pay your housing loan monthly instalments for HDB flats. Your eligibility for an HPS cover is subject to approval and you being in good health. While we are processing your HPS application, you can proceed to use your CPF savings to pay your monthly housing loan instalments.',
          'You can commence your housing monthly deduction by submitting an online application with your Singpass. Please submit your application at least 5 working days before monthly instalment deduction date.',
          'The application process to change your housing monthly deduction depends on the type of loan you have taken.',
          'You can change your housing monthly deduction amount by submitting an online application with your Singpass. Please submit your application at least 5 working days before monthly instalment deduction date.',
          'The application process to stop your housing monthly deduction depends on the type of loan you have taken.',
          'There are two ways for you to stop your housing monthly deduction. Submit an online application at HDB’s website with your Singpass. Upon logging in, look under ‘My Flat > Purchased Flat > Financial Info > Other Related Services > Commence/Vary Monthly CPF deduction. Your application will be processed within three working days upon the receipt of your request from HDB. Complete the CPF withdrawal form (PHS9) at HDB Branch. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system. Your application will be processed within three to five working days upon the receipt of your request from HDB.',
          'You can stop your housing monthly deduction for your housing loan by submitting an online application with your Singpass.',
          'The application process to use your CPF savings for outstanding arrears payment depends on the type of loan you have taken.',
          'There are two ways for you to pay your outstanding arrears payment. Submit an online application at HDB’s website with your Singpass. Upon logging in, look under ‘My Flat > Purchased Flat > Financial Info > Payment of Outstanding instalment by CPF’. Your application will be processed within three working days upon the receipt of your request from HDB. Complete the CPF withdrawal form (PHS9) at HDB Branch. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system. Your application will be processed within three to five working days upon the receipt of your request from HDB.',
          'You can pay your outstanding arrears by submitting an online application with your Singpass. Your application will be processed within three working days.',
          'The application process to use your CPF savings to make partial repayment or full repayment of your housing loan depends on the type of loan you have taken.',
          'Check that you have enough CPF Ordinary Account savings and the maximum amount of CPF savings that you can use for your property.',
          'To make partial or full repayment of your housing loan, you will need to: Seek approval from your housing loan financier. Check that you have enough CPF Ordinary Account savings and the maximum amount of CPF savings that you can use for your property. You can view your Home ownership dashboard to find out your latest usage and usage limit for  your property. Submit an online application with your Singpass. For full repayment of housing loan, you will need to upload a copy of the bank’s redemption statement and a copy of your lawyer’s legal bill (if any) in the application. Your application will be processed within five working days.',
          'The measures available to manage your housing loan monthly instalment depend on who is your financier.',
          'HDB has the following financial assistance measures: Pay for the housing loan arrears by instalments within a reasonable period. Lower the monthly instalment amount by extending the housing loan term up to the maximum repayment period, subject to your eligibility and the 65-year-old age ceiling. Defer or reduce monthly instalments for 6 months. Inclusion of working family members as co-owners to help pay for the monthly instalment/ arrears',
          'You may reach out to your financier for assistance. We have provided the contact information of the financier offering housing loan payment relief (PDF, 32KB) for your convenience.',
          'The process to check your outstanding housing loan depends on the type of loan you have taken.',
          'There are two ways for you to check your outstanding housing loan. Login with your Singpass at HDB’s website with your Singpass. Upon logging in, look under ‘My Flat > Purchased Flat > Financial Info. Visit the HDB Branch managing your flat. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system.',
          'You can check with your bank on your outstanding loan.',
          'You may check with your financier directly. You are advised to pay up as soon as possible as your financier will charge a penalty fee for late or non-payment of instalment.',
          'As your Special Account (SA) savings are meant for retirement, only your Ordinary Account (OA) savings can be used to make a lump sum payment for your property.',
          'On your 55th birthday, a Retirement Account (RA) will be created for you. Savings up to your Full Retirement Sum (FRS) from your Special Account (SA) and Ordinary Account (OA) will be transferred to your Retirement Account (RA). You can continue to use the following from your CPF savings to pay your housing loan: Your savings which you have applied to reserve in your Ordinary Account (OA) before your 55th birthday. This reserved amount 1  will not be transferred to your Retirement Account (RA). Hence, you can continue to use it to pay your housing loan after turning 55 years old. To reserve your Ordinary Account (OA) savings, please log on using your Singpass to my cpf Online Services > Click on “My Requests” > Select “Retirement (eg. Retirement Sum & Age 55 applications)” > Select "Decide on my CPF options for members reaching 55"; Your new CPF contributions to your Ordinary Account (OA) (if you continue working after 55). 1 When you start your monthly payouts under CPF Life or the Retirement Sum Scheme, your reserved OA savings will be transferred to your Retirement Account (RA) if you have not set aside your Full Retirement Sum (FRS). This transfer increases your RA savings and will enhance your monthly payouts.',
          'To reserve your OA savings to pay for your existing property, please log on using your Singpass to my cpf Online Services > Click on “My Requests” > Select “Retirement (eg. Retirement Sum & Age 55 applications)” > Select "Decide on my CPF options for members reaching 55” If you do not have an existing property but may purchase one in the future, you can still reserve your OA savings now for your expected future housing needs. Please submit your request via Write to us. Although you need not submit any documents now, please provide your Option to Purchase or Sale and Purchase Agreement separately after you have committed to a property purchase. If you have already committed to a property purchase, please attach a copy of your Option to Purchase or Sale and Purchase Agreement in your request. Your request should reach the Board at least 3 weeks before your 55th birthday. Please note that when you start your monthly payouts under CPF Life or the Retirement Sum Scheme, your reserved OA savings will be transferred to your Retirement Account (RA) if you have not set aside your Full Retirement Sum (FRS). This transfer increases your RA savings and will enhance your monthly payouts.',
          'You can request to reserve all or part of your Ordinary Account (OA) savings. When you start your monthly payouts under CPF LIFE or the Retirement Sum Scheme, your reserved OA savings will be transferred to your RA if you have not set aside your Full Retirement Sum (FRS). This transfer increases your RA savings and will enhance your monthly payouts.',
          'It will affect the amount that you can withdraw as the Ordinary Account savings reserved for your housing payments cannot be withdrawn in cash. As the Ordinary Account savings reserved for your housing payments would not be transferred to your Retirement Account to make up your retirement sum, this may also affect your monthly payouts when you reach your payout eligibility age.',
          'To help members plan ahead, members who intend to reserve their OA savings for housing payments beyond age 55 may submit a request to CPF Board before the monies are transferred to the RA. This applies to members who own an existing property, as well as those who have yet to purchase a property. Your request should reach the Board at least 3 weeks before your 55th birthday. However, when you start your monthly payouts under CPF Life or the Retirement Sum Scheme, your reserved OA savings will be transferred to your RA if you have not set aside your Full Retirement Sum. This transfer increases your RA savings and will enhance your monthly payouts. Find out how to apply to reserve your OA savings for your housing payment.',
          'You can view your Home ownership dashboard to check the amount to be refunded to your CPF account upon the sale of your property.',
          'You do not have to top up your Retirement Account as you have never used your CPF savings for the property purchase.',
          'You do not have to refund the CPF savings used by your late parent(s) and the accrued interest to their CPF accounts.',
          'The CPF savings that you had previously withdrawn for your property did not form part of the amount approved for withdrawal of CPF on Reduced Life Expectancy. Therefore, upon the sale of your property, the CPF principal amount withdrawn for the property and the accrued interest will need to be refunded to your CPF account. You can apply to withdraw the refunded amount under Reduced Life Expectancy if you are eligible to.',
          'You may withdraw further CPF savings from your Special and Ordinary accounts if there is no change in your medical condition. You can submit an online request for us to assess your eligibility for further withdrawal.',
          'When you sell your property, you will have to refund the principal CPF withdrawn towards the property (including the CPF Housing Grant) plus its accrued interest to your CPF account. Generally, the housing grant will be refunded to your Ordinary Account. If you have received more than $30,000 in housing grants, part of the housing grants may be credited to your Special Account/Retirement Account and MediSave Account. You may view your Home ownership dashboard to see the amount that will be credited to your Special/Retirement Account and MediSave Account. These savings that are credited to your Special Account/Retirement Account and MediSave Account can then be used for approved purposes under the various CPF approved schemes to help you with your healthcare and retirement needs.',
          'Interest will accrue on the amount of CPF savings that you have used for your property. You are required to refund the accrued interest together with the CPF principal amount withdrawn when you sell, transfer or dispose your property.',
          'Accrued interest is the interest amount that you would have earned if your CPF savings had not been withdrawn for housing. The interest is computed on the CPF principal amount withdrawn for housing on a monthly basis (at the current CPF Ordinary Account interest rate) and compounded yearly. You can view the applicable CPF interest rates for the various periods.',
          'You will still need to refund the CPF principal amount withdrawn and the accrued interest even though you are aged 55 and above and have set aside your Retirement Sum. If the selling price (including the option monies) after paying the outstanding housing loan is not enough to fully refund the CPF principal amount withdrawn together with the accrued interest, you do not need to top up the shortfall in cash, provided the property is sold at market value. However, any option monies (e.g. option fee and option exercise fee) received from your buyer in cash upon the sale of your property are considered part of the selling price and need to be refunded to your co-owner\'s and your CPF accounts before the transaction can be completed. The amount refunded from the sale of the property will be used to meet your retirement savings up to the Full Retirement Sum (FRS) in your Retirement Account. Thereafter, any balance will be paid to you in cash. If you would like to: Only refund up to your FRS so that you can receive the balance housing refund without waiting for the pay-out, you can write to us two weeks before the completion of the sale of your property; or Keep the balance refunds in your Ordinary Account to pay for another property, you can submit a request via My Mailbox two weeks before the completion of the sale of your property',
          'If you are aged 55 and above and would like to update the payment mode to receive your excess housing refunds,  please submit an online application via my cpf Online Services - My Requests using your Singpass at least two weeks before the completion of your property sale. To better protect members from scams, we will only accept bank accounts where the bank is able to provide automated bank account verification to CPF Board. This ensures that payment will only be made to a bank account belonging to the member. Currently, only POSB, DBS, OCBC and UOB are able to provide this service. We welcome any other bank to set up similar arrangements with us. The same bank account will also be used to credit the Account for any payment due to you: under the CPF Act and its subsidiary legislation; and from the Government in respect of which CPF Board is the administrator, agent or trustee, to the Account.',
          'Once the excess housing refunds are credited into your CPF account upon the sale of your property, you will receive the refunds in your bank account (maintained in our records) within five working days. If there are no bank account records maintained with us, you will receive a cheque about 10 working days after the housing refunds are credited into your CPF account.',
          'The information on your selling price and outstanding loan were not available when the letter was sent to you. If you are aged 55 and above, we will inform you if you do not have any balance housing refunds after the top up to your Retirement Account, upon completion of the sale of your property.',
          'Your ex-spouse will have to refund the CPF principal amount he/she has withdrawn for the property including accrued interest when he/she is no longer an owner of the property. However, the CPF laws were revised from 1 October 2007 to allow the Court to have the flexibility to make an order for the property to be transferred from a member to his/her ex-spouse without requiring the full CPF refund to be made into the member\'s CPF account in respect of the moneys withdrawn by him/her at the point of transfer. Should the ex-spouse sell the property in future, he/she is required to refund into his/her own CPF account the moneys withdrawn from the member\'s CPF account (which were not refunded upon transfer) as well as the principal amount withdrawn from his/her own CPF account for the purchase of the property with accrued interest. It would be for the Court to decide whether court orders extracted before 1 October 2007 could be amended or varied to benefit from the existing CPF laws. In this respect, please consult your lawyers.',
          'To discharge the CPF charge, you need to refund the amount used for the property and the accrued interest to your CPF account. You can do so by: Voluntarily refunding the amount used to your CPF account. After the refund is made, you will need to engage a lawyer to lift the CPF charge on your property. Please note that you will have to pay any legal cost incurred for this discharge. If you are above 55 and wish to withdraw your Retirement Account savings above the Basic Retirement Sum by pledging the property in future, a fresh CPF charge will be lodged to secure the refund of the withdrawal amount when the property is sold/transferred in future. Please note that the cost of lodgement of charge will be borne by you. Refunding the amount used to your CPF account when you sell your property. Your lawyers handling the sale will complete the discharge as part of the sale transaction.',
          'Upon receipt of the funds, we will process the refunds within five working days for the sale of property. This includes the time needed to clear the cheque/cashier\'s order.',
          'You will need to check with HDB on your eligibility to sell or transfer your HDB flat. After HDB has given their consent, HDB or your lawyer (if you have one acting on your behalf) will check with the Board on the amount (if any) to be refunded to your CPF account.',
          'You will need to instruct your lawyer to obtain consent from the Board before you sell or transfer your private property.',
          'Please confirm if you are selling your whole property, transferring or selling your share of the property to another party in order to determine the amount you need to refund to your CPF account.',
          'You will need to refund the CPF principal amount withdrawn plus the accrued interest ("P+I"). If you have attained the age of 55 and have pledged the property to make up your retirement sum, you will need to refund the pledged amount on top of the P+I. The amount refunded will be used to top up your Retirement Account, up to your Full Retirement Sum. After this, any balance housing refunds will be paid to you in cash. If the selling price (including the option monies) after paying the outstanding housing loan is not enough to fully refund your required CPF refund, you do not need to top up the shortfall in cash, provided that the property is sold at market value. However, the option monies (e.g. option fee and option exercise fee) received from your buyer in cash upon the sale of your property are considered part of the selling price and need to be refunded to your co-owner\'s and your CPF accounts before the transaction can be completed.',
          'You will need to refund the CPF principal amount withdrawn plus the accrued interest ("P+I").  If you have attained the age of 55 and have pledged the property to make up your retirement sum, you will need to refund the pledged amount on top of the P+I. The amount refunded will be used to top up your Retirement Account, up to your Full Retirement Sum. After this, any balance housing refunds will be paid to you in cash.',
          'You will need to refund the CPF principal amount withdrawn plus the accrued interest ("P+I"). If you have attained the age of 55 and have pledged the property to make up your retirement sum, you will need to refund the pledged amount on top of the P+I. The amount refunded will be used to top up your Retirement Account, up to your Full Retirement Sum. After this, any balance housing refunds will be paid to you in cash. If the part share selling price less your share of the outstanding housing loan is insufficient to fully refund your required CPF refund, and provided that the part share transaction is conducted at market value, your will need to refund the higher of the below two figures, capped at your P+I* *If you have pledged the property to make up your retirement sum, the amount will be capped at the sum of P+I and the pledged amount.',
          'Generally, the selling price will be used to pay for the following in this order: Outstanding housing loan, Required CPF refund, Other sale expenses, e.g., legal fees. If the selling price (including the option monies) after paying the outstanding housing loan is not enough to make the required CPF refund, you do not need to top up the shortfall in cash, provided the property is sold at market value. However, the option monies (e.g. option fee and option exercise fee) received from your buyer in cash upon the sale of your property are considered part of the selling price and need to be refunded to your co-owner’s and your CPF accounts before the transaction can be completed.',
          'The selling price will be used to pay for the following in this order: CPF principal amount withdrawn up to 80% of the lower of the purchase price or valuation price of the property at the time of purchase plus the CPF savings used for the legal and stamp fees. On equal ranking: CPF principal amount withdrawn for the remaining 20% of the lower of the purchase price or valuation price of the property at the time of purchase. Bank\'s outstanding housing loan and interest. CPF principal amount withdrawn beyond the lower of the purchase price or valuation price of the property at the time of purchase and the accrued interest. If the sale proceeds (including the option monies) are not enough to make up the required CPF refund and the outstanding housing loan, the sale proceeds will be used to settle the required CPF refund and the outstanding housing loan in the order agreed by you, your bank and the Board. Any option monies (e.g. option fee and option exercise fee) received from your buyer in cash upon the sale of your property are considered part of the selling price and need to be refunded to your co-owner\'s and your CPF accounts before the transaction can be completed.',
          'You can use your Ordinary Account, including the refunded amount, to buy the next property or redeem another housing loan.',
          'You can use your Ordinary Account (OA) savings to buy the next property or redeem another housing loan. The refunds from the sale of your earlier property will be used to top up your Retirement Account, up to your Full Retirement Sum. After this, any balance housing refunds will be paid to you in cash within one week after the CPF refunds are paid into your CPF account. Alternatively, you can request (at least 2 weeks before the completion of the sale of your property) for the balance housing refunds to remain in your OA to pay for the next property or redeem another housing loan.',
          'You will need to refund the CPF principal amount withdrawn plus accrued interest upon sale, transfer or disposal. If the selling price (including the option monies) after paying the outstanding housing loan is not enough to fully refund the CPF principal amount withdrawn together with the accrued interest, you do not need to top up the shortfall in cash, provided the property is sold at market value. However, any option monies (e.g. option fee and option exercise fee) received from your buyer in cash upon the sale of your property are considered part of selling price and need to be refunded to your co-owner\'s and your CPF accounts before the transaction can be completed.',
          'Please check with us on the amount to be refunded upon sale, transfer or disposal.',
          'The CPF Housing Grant (CHG) is disbursed by HDB. Please visit HDB’s website for information on the housing grant and eligibility to apply for CHG.',
          'You can consider taking a bridging loan from any banks so long as Housing & Development Board (HDB) (in the case of HDB flat) and/or your bank allow. The total CPF savings withdrawn plus the loan taken (including bridging loan) cannot exceed the lower of the purchase price or the valuation price of the property at the time of purchase. When your CPF savings are refunded upon the sale of your existing property, you can pay the bridging loan after completing the legal documentation and settling any cash difference.',
          'A joint-tenancy is a form of property ownership where all co-owners have an equal interest in the property, regardless of the individual owner\'s contribution to the property purchase. This means that upon the death of a joint-tenant, his interest in the property will automatically pass on to the remaining co-owner(s), regardless of whether he has made a will.',
          'Please check with Housing & Development Board (HDB) directly on your eligibility to apply for a contra facility to reuse the refunds from the sale of your existing flat to buy another HDB flat.',
          'A tenancy-in-common is a form of property ownership where each co-owner holds a separate and definite share in the property. However, all co-owners are entitled to the enjoyment of the whole property regardless of their share in the property. The tenant-in-common\'s share in the property does not automatically pass on to the remaining co-owner(s). Upon the death of a tenant-in-common, his interest in the flat will be distributed according to his will or if he has no will, it will be distributed according to the provisions of the Intestate Succession Act.',
          'You can use your Ordinary Account (OA) savings to repay the loan taken for the self-construction of your house. The total CPF savings allowed for the construction loan and the existing loan taken to buy the property cannot exceed the new value of the property. You are required to authorise your lawyer (by signing the Letter of Authorisation, Declaration, Consent & Agreement) to submit an online application on your behalf after the issuance of the Temporary Occupation Permit. Your lawyer will need to attach the following documents in the application: Contractors\' breakdown of construction costs; Valuation report of the completed property prepared by a licensed valuer. Valuation report prepared by the financier will be considered on a case by case basis. The Board reserves the right to re-assess the value of the property, if necessary; Original receipts to show evidence of the payment made from your own savings (if you are applying for reimbursement); Outstanding amount of the construction loan; Bank’s Letter of Offer for the construction loan; Bank’s letter of Offer for the conversion of the construction loan to a housing loan; and Temporary Occupation Permit. However, if you already own a property (Housing & Development Board flat or private property) bought with your CPF savings and wish to build another house on a vacant land bought now, you can only use the excess balance in your OA after setting aside the applicable Basic Retirement Sum in your CPF accounts to provide you with a monthly income to support a basic standard of living during retirement. This is provided the remaining lease of the house or your existing property bought with CPF savings can cover you till 95 years old. If the remaining lease of the house or your existing property bought with CPF savings does not cover you till 95 years old, you can only use the excess balance in your OA after setting aside the applicable Full Retirement Sum.',
          'You can use your CPF savings to repay the overdraft, provided it was used solely to buy the private property. You need to authorise your lawyer (by signing the Letter of Authorisation, Declaration, Consent & Agreement) to submit an online application on your behalf. Your lawyer will need to attach the following documents in the application: Financier\'s letter of offer. Financier\'s letter to confirm that the overdraft was used strictly to buy the property or was partially used to buy the property. If the overdraft was used partially to buy the property, please attach the monthly statement of account of the overdraft account from the time it was disbursed for payment of the property to the date of application to use CPF savings. Financier\'s statement showing the current outstanding amount of the overdraft. Once the application is approved, you are required to get your financier to convert the overdraft facility to a housing loan.',
          'The Non-Residential Properties Scheme enabled CPF members to use their CPF savings to buy non-residential properties in Singapore for investment or for their own use. The scheme was stopped on 1 July 2006. The Board will not accept any applications under the scheme now. Members who are already using their CPF savings to service the housing loan for their non-residential properties before 1 July 2006 will not be affected by the policy change.',
          'Non-Residential Properties Scheme (NRPS) was intended to help members return their CPF savings by investing in properties. As CPF members are now allowed to invest in property funds under the CPF Investment Scheme, NRPS is less relevant. Property investment funds offer diversification, are more liquid and do not require large capital commitment. In addition, the take-up rate of NRPS has been low and has also been declining.',
          'CPF savings can only be used to buy residential properties. Small Office Home Office (SOHO) developments are approved by URA either for Office or Residential but not for both uses. You may use your CPF savings to buy a SOHO unit only if URA has approved it for Residential use. Therefore, you are advised to check the approved use of the property before committing to any purchase.',
          'Please complete the CPF withdrawal form (PHS9) at HDB Branch. Do note that services at HDB office and Branches are by appointment only. If you wish to visit HDB Branches, you can book an appointment using the e-Appointment system.      Your application will be processed within three to five working days upon the receipt of your request from HDB.',
          'Please make an appointment to submit your request at any of our service centres. Do note that counter services at CPF Service Centres are available by appointment only. ',
          'To get back your title deed, the procedure depends on the type of property you bought and the loan you took.',
          'You may wish to check with HDB directly on the title deed of your HDB flat.',
          'You may wish to check with your bank directly on the title deed of your HDB flat.',
          'The title deed will only be printed and given to you by the Singapore Land Authority (SLA) if there are no encumbrances on the property such as outstanding mortgages, CPF charges and other charges. The CPF charge was created at the time your CPF savings were withdrawn for the property to secure the refund of CPF principal amount withdrawn including accrued interest upon the sale of the property. If there is still a CPF charge on the property even though the mortgage and other charges are discharged, SLA will not print the title deed. If you require a copy of the title deed, you may liaise with SLA after the mortgage and/or other charges is/are discharged.'
          ]


.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [3]:
# compute embeddings
corpus_embeddings = sentence_similarity_model.encode(corpus, convert_to_tensor=True)

# store corpus & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'corpus': corpus, 'embeddings': corpus_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

# load corpus & embeddings from disc
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_corpus = stored_data['corpus']
    stored_embeddings = stored_data['embeddings']

### 2. Performs Fact Checking

In [4]:
# Find the closest corpus to input text based on cosine similarity
top_k = min(1, len(stored_corpus))

def factcheck(input):
    result_str = ''

    query_embedding = sentence_similarity_model.encode(str(input), convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, stored_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    result_str ='Best matched FAQ with claim:'

    for score, idx in zip(top_results[0], top_results[1]):
        result_str += '   ' + str(round(score.item(),2)) + '   ' + stored_corpus[idx]

    result_str += ' \n\n'

    # Splits best matched FAQ into sentences
    para_to_sentence = stored_corpus[top_results[1]].split('.')

    # Compute embeddings
    corpus_embeddings = sentence_similarity_model.encode(para_to_sentence)

    # Use cosine-similarity and torch.topk to find highest score
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    result_str += 'Best matched sentence with claim:'

    for score, idx in zip(top_results[0], top_results[1]):
        result_str += '   ' + str(round(score.item(),2)) + '   ' + para_to_sentence[idx]

    result_str += ' \n\n'

    ###### Recognising Textual Entailment (RTE) ######

    premise = para_to_sentence[top_results[1]]

    nli_scores = nli_model.predict([(premise, input)])

    if nli_scores.argmax(axis=1)[0]==2:
        nli_scores = nli_model_v3.predict([(premise, input)])
        #print(nli_scores.argmax(axis=1)[0])

    # converting scores to labels
    nli_label_mapping = ['contradiction', 'entailment', 'neutral']
    nli_labels = [nli_label_mapping[max_score] for max_score in nli_scores.argmax(axis=1)]

    result_str += 'Predicted: ' + str(nli_labels[0])

    return result_str


### 3. Simple User Interface Using Gradio.

Examples you can try out on


1.   You can start CPF LIFE payouts at any age
2.   I will forfeit the remaining amount in my CPF LIFE if I die early
3.   If I don’t have enough money in my RA by 55, I have to top up using cash
4.   I will forfeit the remaining amount in my CPF LIFE if I die early
5.   When you kick the bucket, all our balanced CPF money will go into our NOK CPF Medisave account!
6.   My beneficiary will not be able to get my money after I die. It will be stuck in the CPF.
7.   your CPF monies will be locked up if you don’t withdraw at age 55
8.   CPF monies are returned to the government after I die
9.   CPF LIFE interest not returned to nominees after member's death
10.  I cannot withdraw my CPF money at all
11.  witness must be at least 21 years old and is not your nominee



In [5]:
gr.close_all()

demo = gr.Interface(fn=factcheck,
                    inputs=[gr.Textbox(label="Input your claim:", lines=3)],
                    outputs=[gr.Textbox(label="Result:", lines=3)],
                    title="Automated Fact Checking with S-BERT",
                    #description="Fact checks CPF related claims using `msmarco-distilbert-base-v4` & `cross-encoder/nli-deberta-v3-base` model under the hood!",
                    allow_flagging="never",
                    examples=["You can start CPF LIFE payouts at any age", "I will forfeit the remaining amount in my CPF if I die early", "CPF is a scam", \
                              "CPF LIFE interest not returned to nominees after member's death", "If I don’t have enough money in my RA by 55, I have to top up using cash", \
                              "When you kick the bucket, all our balanced CPF money will go into our NOK CPF Medisave account!", "witness must be at least 21 years old and is not your nominee"]
                    )
# demo.launch(share=True, server_port=int(5775))
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce4c0b6216ffbd0878.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
